![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [47]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import time

# Caminhos atualizados
filename_features = r"C:\Users\Zghos\Desktop\CursoEBAC\CientistaDados\Mod17\UCI HAR Dataset\features.txt"
filename_labels = r"C:\Users\Zghos\Desktop\CursoEBAC\CientistaDados\Mod17\UCI HAR Dataset\activity_labels.txt"

filename_subtrain = r"C:\Users\Zghos\Desktop\CursoEBAC\CientistaDados\Mod17\UCI HAR Dataset\train\subject_train.txt"
filename_xtrain = r"C:\Users\Zghos\Desktop\CursoEBAC\CientistaDados\Mod17\UCI HAR Dataset\train\X_train.txt"
filename_ytrain = r"C:\Users\Zghos\Desktop\CursoEBAC\CientistaDados\Mod17\UCI HAR Dataset\train\y_train.txt"

filename_subtest = r"C:\Users\Zghos\Desktop\CursoEBAC\CientistaDados\Mod17\UCI HAR Dataset\test\subject_test.txt"
filename_xtest = r"C:\Users\Zghos\Desktop\CursoEBAC\CientistaDados\Mod17\UCI HAR Dataset\test\X_test.txt"
filename_ytest = r"C:\Users\Zghos\Desktop\CursoEBAC\CientistaDados\Mod17\UCI HAR Dataset\test\y_test.txt"

# Carregar os nomes das colunas
features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep=r"\s+").squeeze("columns")

# Verificar e renomear colunas duplicadas
if features.duplicated().any():
    print("Duplicatas encontradas em 'features'. Renomeando para evitar problemas.")
    features = features + "_" + features.groupby(features).cumcount().astype(str)

# Verificar novamente se as duplicatas foram removidas
print(features.duplicated().sum(), "duplicatas restantes após renomeação.")

# Carregar os outros dados
labels = pd.read_csv(filename_labels, sep=r"\s+", header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze("columns")
X_train = pd.read_csv(filename_xtrain, sep=r"\s+", header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze("columns")
X_test = pd.read_csv(filename_xtest, sep=r"\s+", header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

Duplicatas encontradas em 'features'. Renomeando para evitar problemas.
0 duplicatas restantes após renomeação.


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [22]:
%%time

# Criar e configurar o modelo
modelo_decision_tree = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)

# Medir o tempo de treinamento
inicio_treino = time.time()
modelo_decision_tree.fit(X_train, y_train)
tempo_treino = time.time() - inicio_treino

# Medir o tempo de predição
inicio_predicao = time.time()
y_train_pred = modelo_decision_tree.predict(X_train)
y_test_pred = modelo_decision_tree.predict(X_test)
tempo_predicao = time.time() - inicio_predicao

# Avaliar acurácia
acuracia_train = accuracy_score(y_train, y_train_pred)
acuracia_test = accuracy_score(y_test, y_test_pred)

# Exibir os resultados
print(f"Acurácia na base de treinamento: {acuracia_train:.4f}")
print(f"Acurácia na base de teste: {acuracia_test:.4f}")
print(f"Tempo de treinamento: {tempo_treino:.4f} segundos")
print(f"Tempo de predição: {tempo_predicao:.4f} segundos")

Acurácia na base de treinamento: 0.9758
Acurácia na base de teste: 0.8799
Tempo de treinamento: 6.1832 segundos
Tempo de predição: 0.0306 segundos
CPU times: total: 4.92 s
Wall time: 6.24 s


### Análise de Desempenho da Árvore de Decisão

#### Acurácia
- **Base de Treinamento**: 97.58%
- **Base de Teste**: 87.99%

A alta acurácia na base de treinamento indica que o modelo consegue ajustar bem os dados fornecidos. Já a acurácia na base de teste, um pouco menor, sugere que o modelo generaliza bem, mas ainda pode estar levemente ajustado aos dados de treinamento, o que pode ser um indício de overfitting leve.

#### Tempo de Processamento
- **Treinamento**: 6.18 segundos
- **Predição**: 0.03 segundos

O tempo de treinamento foi razoável, considerando a complexidade do modelo e o tamanho da base de dados. O tempo de predição foi extremamente rápido, confirmando que árvores de decisão são eficientes para aplicação em tempo real.

#### Utilização da CPU
- **Total**: 4.92 segundos
- **Wall Time**: 6.24 segundos

A maior parte do tempo foi gasta no treinamento, mas ainda assim está dentro de um intervalo aceitável para modelos desta natureza.

## Árvore com PCA

Faça uma análise de componentes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [32]:
# Realizar o PCA para reduzir as variáveis a 1 componente
pca = PCA(n_components=1)

# Transformar os dados
inicio_pca = time.time()
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
tempo_pca = time.time() - inicio_pca

# Criar e configurar o modelo de árvore de decisão
modelo_pca = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)

# Treinar a árvore de decisão com a única componente
inicio_treino = time.time()
modelo_pca.fit(X_train_pca, y_train)
tempo_treino = time.time() - inicio_treino

# Fazer as predições
inicio_predicao = time.time()
y_train_pred_pca = modelo_pca.predict(X_train_pca)
y_test_pred_pca = modelo_pca.predict(X_test_pca)
tempo_predicao = time.time() - inicio_predicao

# Avaliar a acurácia
acuracia_train_pca = accuracy_score(y_train, y_train_pred_pca)
acuracia_test_pca = accuracy_score(y_test, y_test_pred_pca)

# Exibir os resultados
print(f"Tempo de PCA: {tempo_pca:.4f} segundos")
print(f"Tempo de treinamento: {tempo_treino:.4f} segundos")
print(f"Tempo de predição: {tempo_predicao:.4f} segundos")
print(f"Acurácia na base de treinamento: {acuracia_train_pca:.4f}")
print(f"Acurácia na base de teste: {acuracia_test_pca:.4f}")

Tempo de PCA: 0.2581 segundos
Tempo de treinamento: 0.0543 segundos
Tempo de predição: 0.0010 segundos
Acurácia na base de treinamento: 0.4997
Acurácia na base de teste: 0.4571


### Análise de Desempenho com PCA e Árvore de Decisão

#### Tempo de Processamento
- **Tempo para realizar o PCA**: 0.2581 segundos
- **Tempo para treinar o modelo**: 0.0543 segundos
- **Tempo para realizar predições**: 0.0010 segundos

O tempo de processamento total foi extremamente eficiente, especialmente no treinamento e predição, graças à redução significativa do número de variáveis.

#### Acurácia
- **Base de Treinamento**: 49.97%
- **Base de Teste**: 45.71%

A baixa acurácia em ambas as bases (treinamento e teste) indica que **reduzir as variáveis para apenas uma componente principal comprometeu a capacidade do modelo de capturar os padrões presentes nos dados originais**. Isso é esperado, dado que apenas uma fração da variabilidade total foi considerada.

Comparado ao modelo original com todas as variáveis, que alcançou uma acurácia de 87.99% na base de teste, o modelo com PCA apresenta desempenho significativamente inferior.

#### Eficiência de Tempo
A redução para uma única variável explicativa diminuiu consideravelmente o tempo de treinamento e predição, evidenciando que o uso do PCA pode ser vantajoso em situações onde a eficiência é mais importante do que a precisão.

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [40]:
%%time
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import time
import pandas as pd

# Lista de quantidades de componentes principais a serem testadas
quantidades_componentes = [1, 2, 5, 10, 50]

# Dicionário para armazenar os resultados
resultados = []

for n_componentes in quantidades_componentes:
    print(f"\nTestando com {n_componentes} componente(s) principal(is)...")
    
    # Aplicar PCA
    pca = PCA(n_components=n_componentes)
    inicio_pca = time.time()
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    tempo_pca = time.time() - inicio_pca
    
    # Treinar a árvore de decisão
    modelo = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
    inicio_treino = time.time()
    modelo.fit(X_train_pca, y_train)
    tempo_treino = time.time() - inicio_treino
    
    # Fazer predições
    inicio_predicao = time.time()
    y_train_pred = modelo.predict(X_train_pca)
    y_test_pred = modelo.predict(X_test_pca)
    tempo_predicao = time.time() - inicio_predicao
    
    # Avaliar a acurácia
    acuracia_train = accuracy_score(y_train, y_train_pred)
    acuracia_test = accuracy_score(y_test, y_test_pred)
    
    # Armazenar os resultados
    resultados.append({
        "Componentes": n_componentes,
        "Acurácia Treino": acuracia_train,
        "Acurácia Teste": acuracia_test,
        "Tempo PCA": tempo_pca,
        "Tempo Treinamento": tempo_treino,
        "Tempo Predição": tempo_predicao
    })

# Exibir os resultados
resultados_df = pd.DataFrame(resultados)
display(resultados_df)


Testando com 1 componente(s) principal(is)...

Testando com 2 componente(s) principal(is)...

Testando com 5 componente(s) principal(is)...

Testando com 10 componente(s) principal(is)...

Testando com 50 componente(s) principal(is)...


,Componentes,Acurácia Treino,Acurácia Teste,Tempo PCA,Tempo Treinamento,Tempo Predição
0,1,0.499728,0.457075,0.155272,0.058696,0.001542
1,2,0.612758,0.584662,0.146098,0.046571,0.001075
2,5,0.846028,0.788599,0.136379,0.059081,0.000000
3,10,0.892682,0.824228,0.125428,0.162957,0.001647
4,50,0.919342,0.822871,0.158426,0.807651,0.000000


CPU times: total: 2.05 s
Wall time: 1.92 s


### Análise de Desempenho com PCA e Árvore de Decisão

#### Desempenho do Modelo
1. **Acurácia**:
   - O modelo com **1 componente principal** apresenta **baixa acurácia** tanto na base de treinamento quanto na base de teste (49.97% e 45.71%, respectivamente), indicando que uma única componente não consegue capturar adequadamente a variabilidade dos dados.
   - Conforme o número de componentes aumenta, a acurácia melhora significativamente. 
   - Com **50 componentes**, a acurácia na base de treinamento atinge 91.93%, mas a melhora na base de teste estabiliza (82.29%), sugerindo que mais componentes trazem pouca vantagem em generalização.

2. **Generalização**:
   - O modelo com **5 componentes principais** já apresenta uma **boa acurácia de teste (78.86%)**, sendo uma escolha mais eficiente considerando a relação entre desempenho e tempo de processamento.

#### Eficiência de Tempo
1. **PCA**:
   - O tempo de processamento do PCA foi relativamente estável para todas as quantidades de componentes testadas, variando entre **0.1254 e 0.1584 segundos**.

2. **Treinamento**:
   - O tempo de treinamento aumenta conforme o número de componentes cresce. Com **50 componentes**, o tempo sobe para **0.8077 segundos**, enquanto com 5 componentes é de apenas **0.0591 segundos**.

3. **Predição**:
   - O tempo de predição foi consistentemente muito baixo para todas as quantidades de componentes testadas, ficando abaixo de **0.002 segundos**, o que é ideal para aplicações em tempo real.

## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

### Conclusão Final

#### O que aconteceu com a acurácia?
1. **Uso de todas as variáveis**:
   - Com todas as variáveis, a árvore de decisão obteve uma **acurácia na base de teste de 87.99%**, pois o modelo tinha acesso a toda a variabilidade e informações dos dados originais, capturando bem os padrões.

2. **Redução para 1 componente principal**:
   - Quando reduzimos para apenas 1 componente principal com o PCA, a acurácia caiu para **45.71%**. Isso ocorreu porque uma única variável não foi suficiente para explicar os padrões presentes nos dados.

3. **Aumento gradual das componentes principais**:
   - Conforme o número de componentes aumentou:
     - Com **5 componentes**, a acurácia de teste melhorou para **78.86%**.
     - Com **10 componentes**, subiu ainda mais para **82.42%**.
     - Com **50 componentes**, estabilizou em **82.29%**, indicando que adicionar mais variáveis após certo ponto não traz ganhos significativos na generalização.

---

#### O que aconteceu com o tempo de processamento?
1. **Treinamento com todas as variáveis**:
   - O treinamento completo com todas as variáveis levou **6.18 segundos**, devido ao grande número de features que o modelo precisou processar.

2. **Aplicação do PCA**:
   - O tempo do PCA foi insignificante, variando de **0.1254 a 0.1584 segundos**, independentemente do número de componentes testadas.

3. **Tempo de treinamento**:
   - Reduzir o número de componentes com o PCA impactou positivamente o tempo de treinamento:
     - Com **1 componente**, o tempo foi de **0.0587 segundos**.
     - Com **5 componentes**, subiu levemente para **0.0591 segundos**.
     - Com **50 componentes**, aumentou para **0.8077 segundos**, ainda significativamente menor que o treinamento completo.

4. **Tempo de predição**:
   - O tempo de predição foi consistentemente muito baixo, próximo de **0.001 segundos**, em todas as configurações.
     
---

#### Considerações Finais

1. **Acurácia vs. Tempo**:
   - Usar todas as variáveis proporcionou a **melhor acurácia (87.99%)**, mas ao custo de maior tempo de processamento.
   - Usar PCA com **5 a 10 componentes** ofereceu um bom equilíbrio entre **desempenho (acurácia acima de 78%)** e **eficiência de processamento**.

2. **Observações**:
   - Para priorizar **eficiência de tempo**, reduzir para **5 componentes** é a melhor opção.
   - Para garantir uma **acurácia alta**, usar todas as variáveis ou mais componentes (ex.: 50) é mais apropriado, desde que o tempo de processamento não seja uma restrição.

3. **Aplicação Prática**:
   - Dependendo do cenário, a escolha do número de componentes deve equilibrar precisão e eficiência:
     - **5 componentes** são ideais para modelos rápidos com boa precisão.
     - **Todas as variáveis** são recomendadas em casos onde a máxima acurácia é essencial, e o tempo de processamento é secundário.